# XOR 문제 풀기
## 데이터 불러오기 및 구성

In [1]:
# XOR 게이트 파이토치
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score

# 데이터 읽기 함수
def load_dataset(file, device):
    data = np.loadtxt(file)
    print("DATA=", data)
    
    input_features = data[:, 0:-1] # 행은 다 읽고(세로), 열은 하나 빼고(y빼고)
    print("INPUT_FEATURES=", input_features)
    
    labels = np.reshape(data[:, -1],(4,1)) # 레아블 형태를 맞추기 위해 reshape
    print("LABELS=", labels)
    
    input_features = torch.tensor(input_features, dtype=torch.float).to(device) # 토치 텐서 형태로 
    labels = torch.tensor(labels, dtype=torch.float).to(device)
    
    return (input_features, labels)

# 모델 평가 결과 계산을 위해 텐서를 리스트로 변환하는 함수
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

## device설정 및 multiple perceptron으로 학습해보기

In [2]:
if torch.cuda.is_available():
    device = 'cuda' # 맥이라 CPU로 처리
else:
    device = 'cpu'
    
input_features,  labels = load_dataset('train.txt', device)

# NN 모델 만들기
model = nn.Sequential(
    nn.Linear(2,2,bias=True), nn.Sigmoid(),
    nn.Linear(2,1, bias=True), nn.Sigmoid()).to(device)

loss_func = torch.nn.BCELoss().to(device) # cost 함수
optimizer = torch.optim.SGD(model.parameters(),lr=1) # 옵티마이저 설정(역전파 알고리즘 수행 함수) -> W: W - lr*cost(W)W로 편미분한 값 // 통해 가중치 업데이트

model.train() # 학습

DATA= [[0. 0. 0.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
INPUT_FEATURES= [[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
LABELS= [[0.]
 [1.]
 [1.]
 [0.]]


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=2, out_features=1, bias=True)
  (3): Sigmoid()
)

In [ ]:
for epoch in range(1001):
    # 기울기 계산한 것들 초기화
    optimizer.zero_grad()
    
    # H(X) 계산: forward 연산
    hypothesis = model(input_features)
    
    # 비용 계산
    cost = loss_func(hypothesis, labels)
    
    # 역전파 수행
    cost.backward()
    optimizer.step() # 옵티마이저 수행
    
    # 100에포크 마다 비용 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())
        
# 평가모드 셋팅(학습 시에 적용했던 드랍 아웃 여부 등을 비적용)
model.eval()

# 역전파를 적용하지 않도록 context manager 설정
with torch.no_grad():
    hypothesis = model(input_features)
    logits = (hypothesis > 0.5).float() # 1과 0으로 리턴. 그런 값을 logit이라고 함
    predicts = tensor2list(logits)
    golds = tensor2list(labels)
    print('PRED=', predicts)
    print("GOLD", golds)
    print('ACC:{0:f}'.format(accuracy_score(golds, predicts)))
    
    # 갈수록 에러가 줄어들고, 정확도는 1이 출력 됨

0 0.7548189163208008
100 0.6916934847831726
200 0.6849790811538696
300 0.6260126829147339
400 0.49085718393325806
500 0.1589161902666092
600 0.06807442009449005
700 0.041601963341236115
800 0.029603593051433563
900 0.022858578711748123
1000 0.018565798178315163
PRED= [[0.0], [1.0], [1.0], [0.0]]
GOLD [[0.0], [1.0], [1.0], [0.0]]
ACC:1.000000


## 은닉 노드 추가

In [4]:
# NN모델 만들기
# 2*2에서 2*10로 변경
model = nn.Sequential(
    nn.Linear(2,10,bias=True), nn.Sigmoid(),
    nn.Linear(10,1, bias=True), nn.Sigmoid()).to(device)
loss_func = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=1)

model.train()

Sequential(
  (0): Linear(in_features=2, out_features=10, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=10, out_features=1, bias=True)
  (3): Sigmoid()
)

In [ ]:
for epoch in range(1001):
    # 기울기 계산한 것들 초기화
    optimizer.zero_grad()
    
    # H(X) 계산: forward 연산
    hypothesis = model(input_features)
    
    # 비용 계산
    cost = loss_func(hypothesis, labels)
    
    # 역전파 수행
    cost.backward()
    optimizer.step()
    
    # 100에폭마다 비용 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())
        
# 평가모드 셋팅(학습 시에 적용했던 드랍 아웃 여부 등을 비적용
model.eval()

# 역전파를 적용하지 않도록 context manager 설정
with torch.no_grad():
    hypothesis = model(input_features)
    logits = (hypothesis > 0.5).float()
    predicts = tensor2list(logits)
    golds = tensor2list(labels)
    print('PRED=', predicts)
    print("GOLD", golds)
    print('ACC:{0:f}'.format(accuracy_score(golds, predicts)))
    
# 학습속도는 쪼금 더 느리지만, 더 빨리 수렴. widening은 선의 개수를 늘리는 효과를 지님

0 0.6943934559822083
100 0.688876748085022
200 0.6416378021240234
300 0.416054904460907
400 0.1545761078596115
500 0.06947584450244904
600 0.040344394743442535
700 0.027117617428302765
800 0.019914666190743446
900 0.015499919652938843
1000 0.012563936412334442
PRED= [[0.0], [1.0], [1.0], [0.0]]
GOLD [[0.0], [1.0], [1.0], [0.0]]
ACC:1.000000


## single layer perceptron으로 해보기

In [9]:
# Hidden layer를 없애고 Single-layer Perceptron으로 변경
model = nn.Sequential(
    nn.Linear(2,1, bias=True), nn.Sigmoid()).to(device)
loss_func = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=1)

model.train()

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
  (1): Sigmoid()
)

In [10]:
for epoch in range(10001):
    # 기울기 계산한 것들 초기화
    optimizer.zero_grad()
    
    # H(X) 계산: forward 연산
    hypothesis = model(input_features)
    
    # 비용 계산
    cost = loss_func(hypothesis, labels)
    
    # 역전파 수행
    cost.backward()
    optimizer.step()
    
    # 100에폭마다 비용 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())
        
# 평가모드 셋팅(학습 시에 적용했던 드랍 아웃 여부 등을 비적용
model.eval()

# 역전파를 적용하지 않도록 context manager 설정
with torch.no_grad():
    hypothesis = model(input_features)
    logits = (hypothesis > 0.5).float()
    predicts = tensor2list(logits)
    golds = tensor2list(labels)
    print('PRED=', predicts)
    print("GOLD", golds)
    print('ACC:{0:f}'.format(accuracy_score(golds, predicts)))
    
    # 정확도 1이 안나옴. non-linear separable 문제여서 단층으로 못품

0 0.8262089490890503
100 0.6931471824645996
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996
700 0.6931471824645996
800 0.6931471824645996
900 0.6931471824645996
1000 0.6931471824645996
1100 0.6931471824645996
1200 0.6931471824645996
1300 0.6931471824645996
1400 0.6931471824645996
1500 0.6931471824645996
1600 0.6931471824645996
1700 0.6931471824645996
1800 0.6931471824645996
1900 0.6931471824645996
2000 0.6931471824645996
2100 0.6931471824645996
2200 0.6931471824645996
2300 0.6931471824645996
2400 0.6931471824645996
2500 0.6931471824645996
2600 0.6931471824645996
2700 0.6931471824645996
2800 0.6931471824645996
2900 0.6931471824645996
3000 0.6931471824645996
3100 0.6931471824645996
3200 0.6931471824645996
3300 0.6931471824645996
3400 0.6931471824645996
3500 0.6931471824645996
3600 0.6931471824645996
3700 0.6931471824645996
3800 0.6931471824645996
3900 0.6931471824645996
4000 0.6931471824645996
4100 0.6931471824645996
4200

## 다시 multiple로 하고 층을 많이 쌓아서 기울기 소실 문제 실험

In [17]:
## Hidden layer층을 1개에서 2개로 변경
## Deeping은 선을 구부리는 효과

model = nn.Sequential(
    nn.Linear(2,2, bias=True), nn.Sigmoid(),
    nn.Linear(2,2, bias=True), nn.Sigmoid(),
    nn.Linear(2,1, bias=True), nn.Sigmoid()).to(device)
loss_func = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=1)

model.train()

for epoch in range(3000):
    # 기울기 계산한 것들 초기화
    optimizer.zero_grad()
    
    # H(X) 계산: forward 연산
    hypothesis = model(input_features)
    
    # 비용 계산
    cost = loss_func(hypothesis, labels)
    
    # 역전파 수행
    cost.backward()
    optimizer.step()
    
    # 500에폭마다 비용 출력
    if epoch % 500 == 0:
        print(epoch, cost.item())
        
# 평가모드 셋팅(학습 시에 적용했던 드랍 아웃 여부 등을 비적용
model.eval()

# 역전파를 적용하지 않도록 context manager 설정
with torch.no_grad():
    hypothesis = model(input_features)
    logits = (hypothesis > 0.5).float()
    predicts = tensor2list(logits)
    golds = tensor2list(labels)
    print('PRED=', predicts)
    print("GOLD", golds)
    print('ACC:{0:f}'.format(accuracy_score(golds, predicts)))

0 0.7609161734580994
500 0.693142294883728
1000 0.6929212808609009
1500 0.6818122863769531
2000 0.028176279738545418
2500 0.006109245121479034
PRED= [[0.0], [1.0], [1.0], [0.0]]
GOLD [[0.0], [1.0], [1.0], [0.0]]
ACC:1.000000


In [32]:
# Hidden layer층을 7개로 변경

model = nn.Sequential(
    nn.Linear(2,10, bias=True), nn.Sigmoid(),
    nn.Linear(10,10, bias=True), nn.Sigmoid(),
    nn.Linear(10,10, bias=True), nn.Sigmoid(),
    nn.Linear(10,10, bias=True), nn.Sigmoid(),
    nn.Linear(10,10, bias=True), nn.Sigmoid(),
    nn.Linear(10,10, bias=True), nn.Sigmoid(),
    nn.Linear(10,1, bias=True), nn.Sigmoid()).to(device)
loss_func = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=1)

model.train()

for epoch in range(10000):
    # 기울기 계산한 것들 초기화
    optimizer.zero_grad()
    
    # H(X) 계산: forward 연산
    hypothesis = model(input_features)
    
    # 비용 계산
    cost = loss_func(hypothesis, labels)
    
    # 역전파 수행
    cost.backward()
    optimizer.step()
    
    # 2500에폭마다 비용 출력
    if epoch % 2500 == 0:
        print(epoch, cost.item())
        
# 평가모드 셋팅(학습 시에 적용했던 드랍 아웃 여부 등을 비적용
model.eval()

# 역전파를 적용하지 않도록 context manager 설정
with torch.no_grad():
    hypothesis = model(input_features)
    logits = (hypothesis > 0.5).float()
    predicts = tensor2list(logits)
    golds = tensor2list(labels)
    print('PRED=', predicts)
    print("GOLD", golds)
    print('ACC:{0:f}'.format(accuracy_score(golds, predicts)))

# 더 많이 쌓아봤지만, 정확도가 낮아짐 -> 기울기 소실 문제

0 0.7029804587364197
2500 0.6931471228599548
5000 0.6931471824645996
7500 0.6931471228599548
PRED= [[1.0], [0.0], [1.0], [0.0]]
GOLD [[0.0], [1.0], [1.0], [0.0]]
ACC:0.500000


## ReLU 사용(기울기 소실 문제 해결!)

In [39]:
# 기울기 소실
# Hidden layer층을 7개로 변경

model = nn.Sequential(
    nn.Linear(2,10, bias=True), nn.ReLU(),
    nn.Linear(10,10, bias=True), nn.ReLU(),
    nn.Linear(10,10, bias=True), nn.ReLU(),
    nn.Linear(10,10, bias=True), nn.ReLU(),
    nn.Linear(10,10, bias=True), nn.ReLU(),
    nn.Linear(10,10, bias=True), nn.ReLU(),
    nn.Linear(10,1, bias=True), nn.Sigmoid()).to(device)
loss_func = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=1)

model.train()

for epoch in range(3000):
    # 기울기 계산한 것들 초기화
    optimizer.zero_grad()
    
    # H(X) 계산: forward 연산
    hypothesis = model(input_features)
    
    # 비용 계산
    cost = loss_func(hypothesis, labels)
    
    # 역전파 수행
    cost.backward()
    optimizer.step()
    
    # 1000에폭마다 비용 출력
    if epoch % 1000 == 0:
        print(epoch, cost.item())
        
# 평가모드 셋팅(학습 시에 적용했던 드랍 아웃 여부 등을 비적용
model.eval()

# 역전파를 적용하지 않도록 context manager 설정
with torch.no_grad():
    hypothesis = model(input_features)
    logits = (hypothesis > 0.5).float()
    predicts = tensor2list(logits)
    golds = tensor2list(labels)
    print('PRED=', predicts)
    print("GOLD", golds)
    print('ACC:{0:f}'.format(accuracy_score(golds, predicts)))

0 0.7003188133239746
1000 1.6257423339993693e-05
2000 5.990302270220127e-06
PRED= [[0.0], [1.0], [1.0], [0.0]]
GOLD [[0.0], [1.0], [1.0], [0.0]]
ACC:1.000000


## 드롭아웃 넣어보기

In [48]:
# 기울기 소실
# Hidden layer층을 7개로 변경

model = nn.Sequential(
    nn.Linear(2,10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10,10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10,10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10,10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10,10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10,10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10,1, bias=True), nn.Sigmoid()).to(device)
loss_func = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.2)

model.train()

for epoch in range(3000):
    # 기울기 계산한 것들 초기화
    optimizer.zero_grad()
    
    # H(X) 계산: forward 연산
    hypothesis = model(input_features)
    
    # 비용 계산
    cost = loss_func(hypothesis, labels)
    
    # 역전파 수행
    cost.backward()
    optimizer.step()
    
    # 1000에폭마다 비용 출력
    if epoch % 1000 == 0:
        print(epoch, cost.item())
        
# 평가모드 셋팅(학습 시에 적용했던 드랍 아웃 여부 등을 비적용)
model.eval()

# 역전파를 적용하지 않도록 context manager 설정
with torch.no_grad():
    hypothesis = model(input_features)
    logits = (hypothesis > 0.5).float()
    predicts = tensor2list(logits)
    golds = tensor2list(labels)
    print('PRED=', predicts)
    print("GOLD", golds)
    print('ACC:{0:f}'.format(accuracy_score(golds, predicts)))

0 0.7124330401420593
1000 0.17117588222026825
2000 0.039803288877010345
PRED= [[0.0], [1.0], [1.0], [0.0]]
GOLD [[0.0], [1.0], [1.0], [0.0]]
ACC:1.000000
